In [1]:
import pandas as pd
import plotly.graph_objects as go

# Load the datasets
df_emission = pd.read_csv("../notebooks/cache/Schiphol_Emissie_Cleaned.csv")
df_yearly = pd.read_csv("../notebooks/cache/Yearly_Traffic_Data_Cleaned.csv")

# Preprocess the emission dataset
df_emission['EMISSIE (kg)'] = df_emission['EMISSIE (kg)'].str.replace(',', '.').astype(float)

# Filter the data for "Koolstofdioxide"
df_koolstofdioxide = df_emission[df_emission['STOFNAAM'] == 'Koolstofdioxide']

# Aggregate the data by year
df_aggregated = df_koolstofdioxide.groupby('EMISSIEJAAR')['EMISSIE (kg)'].sum().reset_index()
df_aggregated.rename(columns={'EMISSIEJAAR': 'Year', 'EMISSIE (kg)': 'CO2 Emissions (kg)'}, inplace=True)

# Merge with the yearly traffic data
df_merged = pd.merge(df_yearly, df_aggregated, how='inner', left_on='Year', right_on='Year')

# Calculate the yearly growth percentages
df_merged['Flight Movements Growth (%)'] = df_merged['Air_Transport_Movements_Total'].pct_change() * 100
df_merged['CO2 Emissions Growth (%)'] = df_merged['CO2 Emissions (kg)'].pct_change() * 100

# Remove the first year since its growth is NaN
df_merged = df_merged.dropna(subset=['Flight Movements Growth (%)', 'CO2 Emissions Growth (%)'])

# Calculate total growth over the entire period
total_flight_movements_growth = (df_merged['Air_Transport_Movements_Total'].iloc[-1] - df_merged['Air_Transport_Movements_Total'].iloc[0]) / df_merged['Air_Transport_Movements_Total'].iloc[0] * 100
total_co2_emissions_growth = (df_merged['CO2 Emissions (kg)'].iloc[-1] - df_merged['CO2 Emissions (kg)'].iloc[0]) / df_merged['CO2 Emissions (kg)'].iloc[0] * 100

# Append total growth to the dataframe
df_total = pd.DataFrame({
    'Year': ['Total'],
    'Flight Movements Growth (%)': [total_flight_movements_growth],
    'CO2 Emissions Growth (%)': [total_co2_emissions_growth]
})

df_merged = pd.concat([df_merged, df_total], ignore_index=True)

# Filter to include only specific years
years_to_include = [2000, 2005, 2010, 2015, 2019, 2020, 2021, 2022, 'Total']
df_merged = df_merged[df_merged['Year'].isin(years_to_include)]

# Create the grouped bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_merged['Year'],
    y=df_merged['Flight Movements Growth (%)'],
    name='Flight Movements Growth (%)',
    marker_color='lightskyblue',
    hovertemplate='%{y:.1f}%<extra></extra>'  # Show only one decimal in the hover info
))

fig.add_trace(go.Bar(
    x=df_merged['Year'],
    y=df_merged['CO2 Emissions Growth (%)'],
    name='CO2 Emissions Growth (%)',
    marker_color='royalblue',
    hovertemplate='%{y:.1f}%<extra></extra>'  # Show only one decimal in the hover info
))

# Update layout to group the bars closely
fig.update_layout(
    title='Yearly Growth in Flight Movements and CO2 Emissions',
    xaxis=dict(
        title='Year',
        type='category',  # Use 'category' type for x-axis
        tickmode='array',
        tickvals=years_to_include,
        tickangle=-45,
        showgrid=False
    ),
    yaxis=dict(
        title="Growth (%)",
        tickmode="array",
        tickvals=[-100, -80, -60, -40, -20, 0, 20, 40, 60, 80, 100],
        ticktext=["-100%", "-80%", "-60%", "-40%", "-20%", "0%", "20%", "40%", "60%", "80%", "100%"],
        showgrid=True
    ),
    barmode='group',
    bargap=0.1,  # Reduce gap between bars within a group
    bargroupgap=0.05,  # Reduce gap between groups
    height=600,
    width=1000,
    plot_bgcolor='white',
    title_font_size=24,
    xaxis_tickfont_size=14,
    yaxis_tickfont_size=14,
    legend=dict(x=0.01, y=1.0, bgcolor='rgba(255,255,255,0)')
)

# Show the plot
fig.show()
